In [18]:
# %matplotlib notebook
import pandas as pd
import numpy as np
import multiprocessing
import argparse
from datetime import timedelta
import pyvips
from PIL import Image
import time
import random
import gc
import logging
import os
from pathlib import Path
import sys
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

sys.path.append('..')
from src.wsi import slides, filters, tiles, util
from src import dataset, dataloading

BASE_PATH = Path(os.path.dirname(os.getcwd()))
DATA_PATH = BASE_PATH / 'data' / 'originals'
PROCESSED_DATA = BASE_PATH / 'data' / 'processed'

In [2]:
low = pd.read_csv(BASE_PATH / 'data' / 'lowres.csv')
hi = pd.read_csv(BASE_PATH / 'data' / 'hires.csv')
# row = train.iloc[2]
# image = pyvips.Image.new_from_file(f'../data/originals/{row["filename"]}', access='sequential')
# image.filename, image.width, image.height

In [3]:
names = !ls ../workspace/tile_data/48
names = [x.split('-')[0] for x in names]

In [13]:
from glob import glob
set1 = set(hi['filename'].str.split('.').str[0])
set2 = set(names)
dels = set2.intersection(set1)
len(dels)
for file in dels:
    f = f'../workspace/tile_data/48/{file}-tile_data.csv'
    os.remove(f)
    f = f'../workspace/tile_data/64/{file}-tile_data.csv'
    os.remove(f)
    f = f'../workspace/tiles/data/processed/*/*/{file}*'
    files = glob(f)
    for fil in files:
        os.remove(fil)

In [3]:
image.get('bands'), image.get('coding'), image.get('format'), image.get('interpretation')

(3, 'none', 'uchar', 'srgb')

In [8]:
image.tiffsave('test.tif', compression="deflate",
                 tile=True, tile_width=256, tile_height=256,
                 pyramid=True)

In [56]:
def rimages(index, train, page):
    filename, res = train.loc[index, ['filename', 'resolution']]
    resizing_factor = res / 0.25
    image = pyvips.Image.new_from_file(f'../data/originals/{filename}', page=page)
    width, height = image.width, image.height
    image = image.affine((resizing_factor, 0, 0, resizing_factor))
    return width, height, image, image.width, image.height

In [3]:
images = pyvips.Image.new_from_file('../data/originals/qmsqv731.tif', page=8)
images.height, images.width
img = images.write_to_file('test.jpg')
# img = np.ndarray(buffer=images.write_to_memory(), dtype=np.uint8, shape=[images.height, images.width, images.bands])
plt.imshow(plt.imread('test.jpg'))


In [10]:
plt.imshow(images.numpy() / 255.)
plt.show()

In [3]:
img = Image.open('test.jpg')
plt.imshow(img)

In [8]:
train.iloc[:, :4].to_csv('../data/train.csv', index=False)

In [87]:
dftrain = dataset.get_dftrain('../workspace/tiles/data/processed/48/3/', 5, 42, 0)
dftrain.dtypes

training dataset: 1252 samples
training dataset: 117941 tiles


tissue_id            object
filename             object
relapse               int64
split                 int64
n_tiles               int64
path                 object
tile_id               int16
is_valid              int64
age                 float64
sex                   int64
melanoma_history      int64
dtype: object

In [79]:
df = dftrain.groupby('tissue_id').agg({'tile_id': list, 'path': list, 'filename': 'first', 'melanoma_history': 'mean', 'sex': 'mean', 'age': 'mean', 'is_valid': 'mean', 'n_tiles': 'mean', 'split': 'mean', 'relapse': 'mean'}).reset_index()
df.dtypes

tissue_id            object
tile_id              object
path                 object
filename             object
melanoma_history    float64
sex                 float64
age                 float64
is_valid            float64
n_tiles             float64
split               float64
relapse             float64
dtype: object

In [88]:
dstrain = dataloading.Dataset('../workspace/tiles/data/processed/48/2/', 5, 42, 0, 80, 16, 256, isval=False)

training dataset: 1252 samples
training dataset: 97504 tiles


In [89]:
dltrain = DataLoader(dstrain, 1, shuffle=True)

In [91]:
image, meta, label = next(iter(dltrain))
meta.dtype

torch.float32

In [4]:
%matplotlib inline

In [12]:
x, y = dls.valid.one_batch()
x.shape

torch.Size([1, 48, 3, 512, 512])